# Model Development

In this notebook, we will develop predictive models using the [Vehicles.csv](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data) dataset. dataset. We will explore various machine learning techniques to create models that can accurately predict used car prices. The process will include building base models and optimizing them through techniques such as regularization and hyperparameter tuning.

By the end of this notebook, we will have established a set of foundational models ready for evaluation, aiming to identify the most effective model for predicting used car prices.